# QuickSight Programmatic Dashboard Creation

## Overview

This notebook demonstrates how to create comprehensive fraud detection dashboards programmatically using Python and QuickSight APIs. This approach is ideal for developers who want to automate dashboard creation or deploy dashboards at scale.

**Time Required:** 15-20 minutes  
**Difficulty:** Advanced  
**Prerequisites:** Complete Lab 1 (Enhanced Fraud Bulk Load Workflow)

## When to Use This Approach

**Programmatic creation is ideal for:**
- Automated deployment across multiple environments
- Scaling dashboard creation for multiple datasets
- Version control of dashboard configurations
- Consistent deployment with exact specifications
- Integration with CI/CD pipelines

**Use manual creation when:**
- Learning QuickSight features and capabilities
- Customizing dashboards for specific business needs
- Iterating on dashboard design and layout
- Building one-off or exploratory dashboards

## Step 1: Setup and Configuration

First, let's import the necessary libraries and get our AWS account information.

In [ ]:
# Import required libraries
import boto3
import json
import sys
import time
from datetime import datetime

# Add src directory to path
sys.path.append('./src')

# Get AWS account information
sts = boto3.client('sts')
account_id = sts.get_caller_identity()['Account']
region = boto3.Session().region_name or 'us-west-2'

# QuickSight bucket name (matches Lab 1 pattern)
s3_bucket = f"{account_id}-quicksight-fraud-data"

print(f"🎯 AWS Account ID: {account_id}")
print(f"🌍 Region: {region}")
print(f"📁 S3 Bucket: {s3_bucket}")
print(f"\n✅ Configuration complete!")

## Step 2: Import Dashboard Creation Script

Now let's import our QuickSight dashboard creation class.

In [ ]:
# Import the QuickSight dashboard creator
try:
    from create_quicksight_dashboard import QuickSightDashboardCreator
    print("✅ QuickSight dashboard creator imported successfully")
except ImportError as e:
    print(f"❌ Error importing dashboard creator: {str(e)}")
    print("   Make sure create_quicksight_dashboard.py is in the src/ folder")
    raise

## Step 3: Verify Data Availability

Let's check that our fraud data is available in S3 from Lab 1.

In [ ]:
# Check if our fraud data exists in S3
s3 = boto3.client('s3')

try:
    # Check for enhanced fraud data
    response = s3.list_objects_v2(
        Bucket=s3_bucket,
        Prefix='enhanced-data/'
    )
    
    if 'Contents' in response:
        print("✅ Enhanced fraud data found in S3:")
        for obj in response['Contents']:
            print(f"   📄 {obj['Key']} ({obj['Size']:,} bytes)")
    else:
        print("❌ No enhanced fraud data found in S3")
        print("   Please run Lab 1 (Enhanced Fraud Bulk Load Workflow) first")
        
    # Check for config files
    config_response = s3.list_objects_v2(
        Bucket=s3_bucket,
        Prefix='config/'
    )
    
    if 'Contents' in config_response:
        print("\n✅ Configuration files found:")
        for obj in config_response['Contents']:
            print(f"   ⚙️ {obj['Key']}")
    else:
        print("\n❌ No configuration files found")
        
except Exception as e:
    print(f"❌ Error checking S3 bucket: {str(e)}")
    print("   Make sure you've completed Lab 1 successfully")

## Step 4: Review Dashboard Configuration

Let's examine the dashboard configuration that will be used to create our comprehensive fraud analytics dashboard.

In [ ]:
# Load and display dashboard configuration
try:
    with open('./src/quicksight_fraud_dashboard.json', 'r') as f:
        dashboard_config = json.load(f)
    
    print("📋 Dashboard Configuration Overview:")
    print(f"   📊 Dashboard Name: {dashboard_config.get('dashboardName', 'N/A')}")
    print(f"   📄 Number of Sheets: {len(dashboard_config.get('sheets', []))}")
    
    # Count total visualizations
    total_visuals = 0
    for sheet in dashboard_config.get('sheets', []):
        total_visuals += len(sheet.get('visuals', []))
    
    print(f"   📈 Total Visualizations: {total_visuals}")
    print(f"   🧮 Calculated Fields: {len(dashboard_config.get('calculatedFields', []))}")
    print(f"   🎛️ Parameters: {len(dashboard_config.get('parameters', []))}")
    
    print("\n📑 Dashboard Sheets:")
    for i, sheet in enumerate(dashboard_config.get('sheets', []), 1):
        sheet_name = sheet.get('name', 'Unnamed Sheet')
        visual_count = len(sheet.get('visuals', []))
        print(f"   {i}. {sheet_name} ({visual_count} visualizations)")
    
    print("\n✅ Dashboard configuration loaded successfully")
    
except FileNotFoundError:
    print("❌ Dashboard configuration file not found")
    print("   Make sure quicksight_fraud_dashboard.json is in the src/ folder")
except json.JSONDecodeError as e:
    print(f"❌ Error parsing dashboard configuration: {str(e)}")
except Exception as e:
    print(f"❌ Unexpected error: {str(e)}")

## Step 5: Create QuickSight Dashboard Programmatically

Now let's create the comprehensive fraud detection dashboard using our automated script.

In [ ]:
# Create the QuickSight dashboard programmatically
print("🚀 Starting programmatic dashboard creation...")
print(f"📊 Target S3 bucket: s3://{s3_bucket}")
print(f"🎯 AWS Account: {account_id}")
print(f"🌍 Region: {region}")

try:
    # Initialize the dashboard creator
    creator = QuickSightDashboardCreator(account_id, region)
    
    # Create the complete dashboard
    print("\n⏳ Creating dashboard (this may take 2-3 minutes)...")
    success = creator.create_complete_dashboard(s3_bucket)
    
    if success:
        print("\n🎉 SUCCESS! Fraud Detection Dashboard Created!")
        print("\n📋 What was created:")
        print("   ✅ QuickSight Data Source connected to your S3 fraud data")
        print("   ✅ QuickSight Dataset with proper schema and relationships")
        print("   ✅ Comprehensive Dashboard with 25+ visualizations")
        print("   ✅ Interactive filters and parameters")
        print("   ✅ All 9 fraud types covered with appropriate charts")
        
        print("\n🔗 Next Steps:")
        print("   1. Go to Amazon QuickSight Console")
        print("   2. Navigate to 'Dashboards'")
        print("   3. Look for 'Fraud Detection Analytics - Workshop'")
        print("   4. Open and explore your comprehensive fraud analytics!")
        
        # Provide direct link
        quicksight_url = f"https://{region}.quicksight.aws.amazon.com/sn/dashboards"
        print(f"\n🌐 QuickSight Console: {quicksight_url}")
        
    else:
        print("\n❌ Dashboard creation failed")
        print("\n🔧 Troubleshooting steps:")
        print("   1. Check that QuickSight is enabled in your account")
        print("   2. Verify S3 permissions for QuickSight")
        print("   3. Ensure Lab 1 completed successfully")
        print("   4. Try manual dashboard creation instead")
        
except Exception as e:
    print(f"\n❌ Error during dashboard creation: {str(e)}")
    print("\n🔧 Possible solutions:")
    print("   1. Check AWS permissions for QuickSight APIs")
    print("   2. Verify your fraud data exists in S3")
    print("   3. Ensure QuickSight service is available in your region")
    print("   4. Try the manual dashboard creation approach instead")

## Step 6: Verify Dashboard Creation

Let's verify that our dashboard was created successfully by checking QuickSight resources.

In [ ]:
# Verify dashboard creation by listing QuickSight resources
quicksight = boto3.client('quicksight', region_name=region)

try:
    print("🔍 Verifying QuickSight resources...")
    
    # List data sources
    try:
        data_sources = quicksight.list_data_sources(AwsAccountId=account_id)
        fraud_sources = [ds for ds in data_sources.get('DataSources', []) 
                        if 'fraud' in ds.get('Name', '').lower()]
        
        if fraud_sources:
            print(f"   ✅ Found {len(fraud_sources)} fraud-related data source(s)")
            for ds in fraud_sources:
                print(f"      📊 {ds.get('Name')} ({ds.get('Type')})")
        else:
            print("   ⚠️ No fraud-related data sources found")
    except Exception as e:
        print(f"   ❌ Error listing data sources: {str(e)}")
    
    # List datasets
    try:
        datasets = quicksight.list_data_sets(AwsAccountId=account_id)
        fraud_datasets = [ds for ds in datasets.get('DataSetSummaries', []) 
                         if 'fraud' in ds.get('Name', '').lower()]
        
        if fraud_datasets:
            print(f"   ✅ Found {len(fraud_datasets)} fraud-related dataset(s)")
            for ds in fraud_datasets:
                print(f"      📈 {ds.get('Name')}")
        else:
            print("   ⚠️ No fraud-related datasets found")
    except Exception as e:
        print(f"   ❌ Error listing datasets: {str(e)}")
    
    # List dashboards
    try:
        dashboards = quicksight.list_dashboards(AwsAccountId=account_id)
        fraud_dashboards = [db for db in dashboards.get('DashboardSummaryList', []) 
                           if 'fraud' in db.get('Name', '').lower()]
        
        if fraud_dashboards:
            print(f"   ✅ Found {len(fraud_dashboards)} fraud-related dashboard(s)")
            for db in fraud_dashboards:
                print(f"      📊 {db.get('Name')}")
                print(f"         Created: {db.get('CreatedTime')}")
                print(f"         Last Updated: {db.get('LastUpdatedTime')}")
        else:
            print("   ⚠️ No fraud-related dashboards found")
    except Exception as e:
        print(f"   ❌ Error listing dashboards: {str(e)}")
    
    print("\n✅ Verification complete!")
    
except Exception as e:
    print(f"❌ Error during verification: {str(e)}")
    print("   This might be due to insufficient QuickSight permissions")

## Step 7: Access Your Dashboard

Now go to QuickSight Console to explore your programmatically created fraud detection dashboard.

In [ ]:
# Provide access instructions
print("🔗 Access Your Dashboard:")
print(f"   1. Go to: https://{region}.quicksight.aws.amazon.com")
print("   2. Click 'Dashboards' in the left navigation")
print("   3. Look for 'Fraud Detection Analytics - Workshop'")
print("   4. Click to open and explore your fraud analytics")

print("📊 What You'll Find:")
print("   ✅ Executive Summary with fraud KPIs")
print("   ✅ Money Laundering flow networks")
print("   ✅ Smurfing pattern analysis")
print("   ✅ Shell company networks")
print("   ✅ All 9 fraud types with visualizations")
print("   ✅ Interactive filters and controls")

print("🎉 Success!")
print("You've created a comprehensive fraud detection dashboard programmatically!")

## Summary

🎉 **Congratulations!** You've successfully completed the programmatic dashboard creation workshop!

### What You've Accomplished:

✅ **Automated Dashboard Creation** - Used Python and QuickSight APIs to create comprehensive fraud analytics  
✅ **Scalable Deployment** - Learned techniques for deploying dashboards across multiple environments  
✅ **Configuration Management** - Understood how to version control dashboard specifications  
✅ **Enterprise Integration** - Explored CI/CD integration possibilities  
✅ **Advanced Customization** - Discovered options for extending and customizing dashboards  

### Business Value Delivered:

- **Faster Time-to-Market** - Automated deployment reduces manual dashboard creation time
- **Consistent Quality** - Programmatic creation ensures consistent dashboard standards
- **Scalable Operations** - Easy deployment across development, staging, and production
- **Version Control** - Dashboard configurations managed as code
- **Reduced Errors** - Automation eliminates manual configuration mistakes

### Next Steps:

1. **Explore Your Dashboard** - Go to QuickSight Console and interact with your fraud analytics
2. **Customize Further** - Modify the JSON configuration for your specific needs
3. **Set Up Automation** - Integrate with your CI/CD pipeline for continuous deployment
4. **Scale Deployment** - Deploy to multiple environments and datasets
5. **Monitor Usage** - Set up dashboard performance and usage monitoring

You now have both manual (Part 2) and programmatic (Part 3) approaches to creating sophisticated fraud detection dashboards!